<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/VisionComputacional/blob/main/L08_Transferencia_de_aprendizaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visión Computacional
## Dr. Carlos Villaseñor
### Lección 8 - Transferencia de aprendizaje

Importar bibliotecas



In [1]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Importar MobileNet base pero descartaremos las últimas capa

In [2]:
base_model=MobileNet(weights='imagenet',include_top=False)
print(base_model.summary())

17227776/17225924 [==============================] - 0s 0us/step
Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
_________________________________________________

Crear nueva Red Neuronal

In [4]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(3, activation='softmax')(x)

Pegar Redes Neuronales

In [5]:
model=Model(inputs=base_model.input, outputs=preds)
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, None, None, 32)    0     

Definir que capas se entrenan

In [6]:
for layer in model.layers[:10]:
    layer.trainable=False
for layer in model.layers[10:]:
    layer.trainable=True

Cargar datos nuevos

- Carge el archivo 'train.zip' en la pestaña archivos en el lateral izquierdo
- Corra la siguiente linea para descomprimir los archivos

In [ ]:
!unzip train.zip 

Crear generador desde carpeta

In [8]:
train_datagen=ImageDataGenerator(zoom_range=0.2, horizontal_flip=True,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('./train/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)



Found 197 images belonging to 3 classes.


Compilar Modelo y entrenar desde carpeta

In [10]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',
              metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=6)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/6
6/6 [==============================] - 4s 384ms/step - loss: 2.5841 - accuracy: 0.4959
Epoch 2/6
6/6 [==============================] - 2s 409ms/step - loss: 0.3369 - accuracy: 0.9056
Epoch 3/6
6/6 [==============================] - 2s 331ms/step - loss: 0.0920 - accuracy: 0.9619
Epoch 4/6
6/6 [==============================] - 2s 330ms/step - loss: 0.3410 - accuracy: 0.9289
Epoch 5/6
6/6 [==============================] - 2s 323ms/step - loss: 0.1743 - accuracy: 0.9470
Epoch 6/6
6/6 [==============================] - 2s 319ms/step - loss: 0.3149 - accuracy: 0.9189
